In [1]:
from owlready2 import *
#from owlready.editor import *
#from owlready.instance_editor import *
import types
import rdflib
from pprint import pprint
import editobj3
from bs4 import BeautifulSoup
import json

In [ ]:
onto = get_ontology("http://test.org/onto.owl")

In [ ]:
def string_cleanup(s):
    s = s.strip()
    problem_chars = ['"', '|', "'", '-', ',', '.', '–']
    for i in problem_chars:
        if i in s:
            s = s.replace(i, '')
    s = s.replace(" ", "_")
    s = s.replace(' ', "_")
    s = s.replace('#', "_")
    s = s.replace("&", "and")
    s = s.replace("+", "plus")
    s = s.replace("%", "percent")
    return s
    
def retrieve_info_from_page(asin):

    item_characteristics_dict = {}
    try:
        api_results = json.load(open('./product_data/' + asin + '.json', 'r'))
        
        for key in api_results:
            if type(api_results[key]) is str:
                item_characteristics_dict[key] = string_cleanup(api_results[key])
            else:
                item_characteristics_dict[key] = api_results[key]
            
    except Exception as e: 
        print(e)
        print(asin)
    

    if asin+'.html' in os.listdir('./pages'):
        
        print('scraping ' + asin) 
        
        page = open('./pages/'+asin+".html", 'r')
        soup = BeautifulSoup(page.read(), 'html.parser')
        tab = soup.find('div', {'id': 'HLCXComparisonWidget_feature_div'})

        # PRODUCT TITLE
        title = soup.find('span', {'id':'productTitle'})
        if title != None:        
            title = string_cleanup(title.text)
            item_characteristics_dict['title'] = title
        else: 
            print("this asin: " + asin + " has problems on title")

        # PRODUCT ASIN
        item_characteristics_dict['asin'] = string_cleanup(asin)

        # PRODUCT CLASSES
        item_characteristics_dict['classes'] = []
        tmp = soup.find('div', {'id':'wayfinding-breadcrumbs_feature_div'})
        if tmp!=None:
            for i in tmp.findAll('a'):
                item_characteristics_dict['classes'].append(string_cleanup(i.text))
        else:
            print("this asin: " + asin + " has problems on classes")

        # PRODUCT BRAND
        if soup.find('a', {'id': 'bylineInfo'}) != None:
            item_characteristics_dict['brand'] = string_cleanup(soup.find('a', {'id': 'bylineInfo'}).text)
        elif soup.find('a', { 'id' : 'brand' }) != None:
            item_characteristics_dict['brand'] = string_cleanup(soup.find('a', { 'id' : 'brand' })['href'].split('/')[1])

        bullets = soup.find('div', {'id':'feature-bullets'})
        #for span in bullets.findAll('span'):
            #print span.text.strip()

        try:
            for i, tr in enumerate(tab.findAll('tr')):
                # tr 0:
                # tr 1:
                # tr 2:
                # tr 3: customer rating
                if i<4: continue

                key = tr.findAll('td')[0].span.text
                if tr.findAll('td')[1].span != None: 
                    value = tr.findAll('td')[1].span.text
                else: 
                    value = tr.findAll('td')[1].a.text
                if key == "Price": 
                    value = value.strip()
                    item_characteristics_dict[key.lower()] = float(value.split('$')[1])   
                else: item_characteristics_dict[string_cleanup(key.lower())] = string_cleanup(value.lower())
        except: 
            print('no tr tab')

        # PROD DETAILS
        details_tab = soup.find('div', {'id':'prodDetails'})

        if soup.find('div', {'id':'productDescription'})!=None: 
            description1 = soup.find('div', {'id':'productDescription'}).p.text
    
    return item_characteristics_dict

In [ ]:
dict_list = []
for i, page in enumerate(os.listdir('./product_data')):
    dict_list.append(retrieve_info_from_page(page[:-5]))
    
json.dump(dict_list, open('tmp.json', 'w'))

In [2]:
onto = get_ontology("http://test.org/onto.owl")
dict_list = json.load(open('tmp.json', 'r'))

In [3]:
for i in onto.properties():
    print(i)

In [4]:
# special properties: classes, brands, sellers
feature_lists = ['features', 'authors', 'reviewTexts', 'languages', 'creators', 'editorial_reviews']
special_features = ['classes', 'brand', 'sold_by', 'title']
feature_dict = {}

"""
for elem in dict_list:
    for key in elem:
        if key not in feature_lists:
            New_Class = types.new_class(key, )
"""

'\nfor elem in dict_list:\n    for key in elem:\n        if key not in feature_lists:\n            New_Class = types.new_class(key, )\n'

In [5]:
# CLASSES
for i in dict_list:
    with onto:
        if "classes" in i:
            prevclass = Thing
            for itemclass in i["classes"]:
                if itemclass not in [j._name for j in onto.classes()]:
                    prevclass = types.new_class(itemclass, (prevclass,))

In [6]:
# BRANDS
with onto:
    if "Brand" not in [j._name for j in onto.classes()]:
        Brand = types.new_class("Brand", (Thing,))
    else:
        Brand = next(j for j in onto.classes() if j.name == "Brand")
    
for i in dict_list:
    with onto:
        if "brand" in i:
            if i["brand"] not in [j._name for j in onto.classes()] and i["brand"] != None:
                New_Class = types.new_class(i['brand'], (Brand, ))                

In [7]:
# SELLERS
with onto:
    if "Seller" not in [j._name for j in onto.classes()]:
        Seller = types.new_class("Seller", (Thing,))
    else:
        Seller = next(j for j in onto.classes() if j.name == "Seller")
    
for i in dict_list:
    with onto:
        if "sold_by" in i:
            if i["sold_by"] not in [j._name for j in onto.classes()]:
                New_Class = types.new_class(i['sold_by'], (Seller, ))                

In [8]:
# PRODUCT GROUPS
with onto:
    for i in dict_list:
        if "product_group" in i:
            if i["product_group"] not in [j._name for j in onto.classes()]:
                 New_Class = types.new_class(i['product_group'], (Thing, ))

In [17]:
# THINGS
for i in dict_list:
    with onto:
        if "title" in i:
            if i["title"] not in [j._name for j in onto.classes()]:
                if "classes" in i and len(i["classes"]) > 0:
                    New_Class1 = types.new_class(i["title"], (next(j for j in onto.classes() if j.name == i["classes"][-1]),))
                elif "product_group" in i and len(i["product_group"]) > 0:
                    New_Class1 = types.new_class(i["title"], (next(j for j in onto.classes() if j.name == i["product_group"]),))
                else:
                    New_Class1 = types.new_class(i["title"], (Thing,))
                
            else: 
                New_Class1 = next(j for j in onto.classes() if j.name == i["title"])

            print("working on " + i["title"])

            # string properties
            for key in i:
                if key not in special_features and type(i[key]) is str:
                    prop_class = types.new_class(key, (DataProperty, ))
                    #prop_class.range = [str]
                    New_Class1.prop_class.append(i[key])


    #         if "color" in i:
    #             has_color = types.new_class("has_color", (DataProperty,))
    #             has_color.range = [str]
    #             New_Class1.has_color.append(str(i["color"]))

    #         if "price" in i:
    #             has_price = types.new_class("has_price", (DataProperty,))
    #             has_price.range = [float]
    #             New_Class1.has_price.append(float(i["price"]))

    #         """
    #         if "brand" in i:
    #             has_brand = types.new_class("has_brand", (ObjectProperty,))
    #             produces = types.new_class("produces", (ObjectProperty,))
    #             has_brand.inverse_property = produces
    #             New_Class1.has_brand.append(next(j for j in onto.classes() if j.name == i["brand"]))
    #         """

    #         if "sold_by" in i:
    #             sold_by = types.new_class("sold_by", (ObjectProperty,))
    #             sells = types.new_class("sells", (ObjectProperty,))
    #             sold_by.inverse_property = sells
    #             New_Class1.sold_by.append(next(j for j in onto.classes() if j.name == i["sold_by"]))

working on HTC_Desire_Eye_E1_16GB_White/Red_GSM_Unlocked_US_Version_(13MP_Front_and_Rear_Camera)


AttributeError: 'prop_class' property is not defined.

In [ ]:
for i in dict_list:
    pprint(i)

In [ ]:
sync_reasoner()
for i in onto.classes():
    print(i.has_color)

In [ ]:
onto.OnePlus_One_A0001_4G_LTE_3GBRAM_64GBROM_55_Inch_International_Version__No_Warranty__Sandstone_Black.has_color

In [ ]:
graph = default_world.as_rdflib_graph()
r = list(graph.query("""SELECT * WHERE { ?a ?b ?c . }"""))
#r = list(graph.query("""SELECT * WHERE { ?a ?b <http://test.org/onto.owl#has_color> }"""))

pprint(r)


#onto.search( has_color = "*")

In [ ]:
onto.save("saved_ontology.rdf")

In [ ]:
camera_set = set()
color_set = set()
for i in dict_list:
    if 'camera description' in i: camera_set.add(i['camera description'])
    if 'color' in i: color_set.add(i['color'])
    
pprint(color_set)

In [ ]:
Phone = types.new_class("Phone", (Thing,), kwds = { "ontology" : onto })
Camera = types.new_class("Camera", (Thing,), kwds = { "ontology" : onto })
Color = types.new_class("Color", (Thing,), kwds = { "ontology" : onto })
has_camera = types.new_class("has_camera", (Property,), kwds = { "ontology" : onto })
has_color = types.new_class("has_color", (Property,), kwds = { "ontology" : onto })
ANNOTATIONS[has_camera]["python_name"] = "has_camera"
ANNOTATIONS[has_color]["python_name"] = "has_color"

In [ ]:
for i in color_set:
    Color(i)

In [ ]:
for i in onto.Color._direct_instances:
    print(vars(i))

In [ ]:
for i in camera_set:
    Camera(i)

In [ ]:
for i in dict_list:
    which_color = ""
    if 'color' not in i: continue
    for j in onto.Color._direct_instances:
        if j._name == i['color']:
            which_color = j
    New_Class = types.new_class(i["asin"][:-5], (Phone,), kwds = { "ontology" : onto })
    New_Class.equivalent_to = [Phone & restriction(has_color, VALUE, j)]

In [ ]:
for i in dict_list:
    

In [ ]:
pprint(onto.instances)

In [ ]:
class Phone(Thing):
    ontology = onto
    
class Camera(Thing):
    ontology = onto
    
class has_camera(Property):
    ontology = onto
    domain = [Phone]
    range = [Camera]
ANNOTATIONS[has_camera]["python_name"] = "has_camera"

camera_8mp = Camera("camera_8mp")
camera_16mp = Camera("camera_16mp")

class galaxy_s8 (Phone):
    ontology = onto
    equivalent_to = [Phone & restriction(has_camera, VALUE, camera_16mp)]
    
    def print_name(self):
        print("hey there")

In [ ]:
pprint(vars(onto.galaxy_s8))

In [ ]:
query = Phone(has_color = [onto.black])
query.closed_world()
onto.sync_reasoner()

In [ ]:
print(vars(query))

In [ ]:
for ontology in onto.indirectly_imported_ontologies(): configure_editobj_from_ontology(ontology)

editobj3.GUI = "Qt" # "Qt" or "HTML"

if editobj3.GUI == "Qt": # Required by Qt
    import sys, PyQt5.QtWidgets as qtwidgets
    if qtwidgets.QApplication.startingUp(): qtwidgets.app = qtwidgets.QApplication(sys.argv)

editor = OntologyInstanceEditor()
editor.set_ontology(onto, edited_classes = onto.classes)
editor.main()